In [189]:
%pip install pandas
%pip install torch
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\mannat2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Goal: Build a classification model to segment the customers based on the account data.

classify the Accounts eligible for Credit Line Increase without Risk, Accounts eligible for Credit Line
Increase but has risk of potential defaults or fraud, No Credit Line Increase required and Non-Performing
accounts that pose a high risk.

# Data Cleaning and Preprocessing

In [213]:
import pandas as pd

In [214]:
account_dim = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/account_dim_20250325.csv")
fraud_claim_case = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/fraud_claim_case_20250325.csv")
fraud_claim_tran = pd.read_csv("C:/Users/mannat2/datathon/consumer_spending_ml/data/tmp/datathon/data/fraud_claim_tran_20250325.csv")
rams_batch_cur = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/rams_batch_cur_20250325.csv")
statement_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/statement_fact_20250325.csv")
syf_id = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/syf_id_20250325.csv")
transfaction_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/transaction_fact_20250325.csv")
wrld_stor_tran_fact = pd.read_csv("C:/Users/mannat2/Documents/GitHub/consumer_spending_ml/data/tmp/datathon/data/wrld_stor_tran_fact_20250325.csv")

In [215]:
rams_batch_cur = rams_batch_cur[['cu_account_nbr', 'cu_bhv_scr', 'cu_crd_bureau_scr', 'ca_avg_utilz_lst_6_mnths', 'ca_avg_utilz_lst_6_mnths', 'mo_tot_sales_array_1', 'mo_tot_sales_array_2', 'mo_tot_sales_array_3', 'mo_tot_sales_array_4', 'mo_tot_sales_array_5', 'mo_tot_sales_array_6', 'cu_cash_line_am', 'cu_crd_line', 'cu_otb', 'cu_nbr_days_dlq', 'ca_max_dlq_lst_6_mnths', 'ca_nsf_count_lst_12_months']]
account_dim = account_dim[['current_account_nbr', 'payment_hist_1_12_mths', 'payment_hist_13_24_mths', 'overlimit_type_flag', 'external_status_reason_code']]
statement_fact = statement_fact[['current_account_nbr', 'return_check_cnt_total']]
fraud_claim_case = fraud_claim_case[['current_account_nbr', 'gross_fraud_amt', 'net_fraud_amt']]
transfaction_fact = transfaction_fact[['current_account_nbr', 'transaction_return_cnt', 'first_purchase_ind']]

In [216]:
rams_batch_cur = rams_batch_cur.rename(columns={"cu_account_nbr": "current_account_nbr"})
fraud_claim_case = fraud_claim_case.rename(columns={"current_account_nbr_pty": "current_account_nbr"})
master_df = account_dim.merge(rams_batch_cur, on='current_account_nbr').merge(statement_fact, on="current_account_nbr").merge(fraud_claim_case, on="current_account_nbr").merge(transfaction_fact, on="current_account_nbr")

In [217]:
master_df.drop(columns=['overlimit_type_flag', 'ca_avg_utilz_lst_6_mnths', 'payment_hist_1_12_mths', 'payment_hist_13_24_mths'], inplace=True)

In [218]:
def first_purchase_ind(row):
    if row['first_purchase_ind'] == 'Y':
        return 1
    else:
        return 0

In [219]:
master_df['first_purchase_ind'] = master_df.apply(first_purchase_ind, axis=1)

In [220]:
master_df

,current_account_nbr,external_status_reason_code,cu_bhv_scr,cu_crd_bureau_scr,mo_tot_sales_array_1,mo_tot_sales_array_2,mo_tot_sales_array_3,mo_tot_sales_array_4,mo_tot_sales_array_5,mo_tot_sales_array_6,...,cu_crd_line,cu_otb,cu_nbr_days_dlq,ca_max_dlq_lst_6_mnths,ca_nsf_count_lst_12_months,return_check_cnt_total,gross_fraud_amt,net_fraud_amt,transaction_return_cnt,first_purchase_ind
0,nTgalbpd5Rsol7CA,0,742,813,0.00,0.00,0.0,0.0,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,1,0
1,nTgalbpd5Rsol7CA,0,742,813,0.00,0.00,0.0,0.0,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,0
2,nTgalbpd5Rsol7CA,0,742,813,0.00,0.00,0.0,0.0,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,0
3,nTgalbpd5Rsol7CA,0,742,813,0.00,0.00,0.0,0.0,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,0,0
4,nTgalbpd5Rsol7CA,0,742,813,0.00,0.00,0.0,0.0,0.0,0.0,...,12900,12900.00,0,0,0,0,557.29,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91929,3hJcpXnkhUx7DBjb,0,745,0,0.00,0.00,0.0,0.0,0.0,0.0,...,2500,2500.00,0,0,0,0,2000.00,0.0,0,0
91930,0ctPdTAguBbpQDMF,0,6,757,0.00,11.87,0.0,0.0,0.0,0.0,...,6500,6474.26,30,1,0,0,11.87,0.0,0,1
91931,0ctPdTAguBbpQDMF,0,6,757,0.00,11.87,0.0,0.0,0.0,0.0,...,6500,6474.26,30,1,0,0,11.87,0.0,0,1
91932,0ctPdTAguBbpQDMF,0,0,786,11.87,0.00,0.0,0.0,0.0,0.0,...,6500,6488.13,0,0,0,0,11.87,0.0,0,1


# Clustering

In [221]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [222]:
master_df.isna().value_counts()

current_account_nbr  external_status_reason_code  cu_bhv_scr  cu_crd_bureau_scr  mo_tot_sales_array_1  mo_tot_sales_array_2  mo_tot_sales_array_3  mo_tot_sales_array_4  mo_tot_sales_array_5  mo_tot_sales_array_6  cu_cash_line_am  cu_crd_line  cu_otb  cu_nbr_days_dlq  ca_max_dlq_lst_6_mnths  ca_nsf_count_lst_12_months  return_check_cnt_total  gross_fraud_amt  net_fraud_amt  transaction_return_cnt  first_purchase_ind
False                False                        False       False              False                 False                 False                 False                 False                 False                 False            False        False   False            False                   False                       False                   False            False          False                   False                 91934
Name: count, dtype: int64

In [223]:
master_df = master_df.groupby('current_account_nbr').agg('mean').reset_index()

In [225]:
standardized_columns = [
    'external_status_reason_code', 'cu_bhv_scr', 'cu_crd_bureau_scr', 'mo_tot_sales_array_1', 
    'mo_tot_sales_array_2', 'mo_tot_sales_array_3', 'mo_tot_sales_array_4', 'mo_tot_sales_array_5', 
    'mo_tot_sales_array_6', 'cu_cash_line_am', 'cu_crd_line', 'cu_otb', 'cu_nbr_days_dlq', 'ca_max_dlq_lst_6_mnths', 
    'ca_nsf_count_lst_12_months', 'return_check_cnt_total', 'gross_fraud_amt', 'net_fraud_amt', 'transaction_return_cnt'
]

In [226]:
scaler = StandardScaler()

In [227]:
master_df_scaled = scaler.fit_transform(master_df[standardized_columns])
master_df_scaled = pd.DataFrame(master_df_scaled, columns=standardized_columns)

In [228]:
master_df[standardized_columns] = master_df_scaled

In [229]:
master_df

,current_account_nbr,external_status_reason_code,cu_bhv_scr,cu_crd_bureau_scr,mo_tot_sales_array_1,mo_tot_sales_array_2,mo_tot_sales_array_3,mo_tot_sales_array_4,mo_tot_sales_array_5,mo_tot_sales_array_6,...,cu_crd_line,cu_otb,cu_nbr_days_dlq,ca_max_dlq_lst_6_mnths,ca_nsf_count_lst_12_months,return_check_cnt_total,gross_fraud_amt,net_fraud_amt,transaction_return_cnt,first_purchase_ind
0,0ctPdTAguBbpQDMF,-0.446446,-1.375831,0.645648,-0.621081,-0.531715,-0.490731,-0.530960,-0.478637,-0.470811,...,-0.058587,0.024878,2.352290,1.778407,-0.145865,-0.145865,-0.534239,-0.394317,-0.492601,1.000000
1,2myqqt3xp0AAjR3b,-0.446446,-0.903619,0.918791,-0.010173,-0.204622,-0.173141,-0.530960,-0.478637,-0.470811,...,-0.898131,-0.933781,3.768384,2.905567,-0.145865,-0.145865,1.164958,0.416182,0.465912,0.100000
2,3Hj6EuxP2uay1tZR,-0.446446,1.055008,-2.301631,-0.630058,-0.540918,-0.490731,-0.530960,-0.478637,-0.470811,...,0.646912,0.768583,-0.479899,-0.475912,-0.145865,-0.145865,-0.214070,-0.394317,-0.492601,0.000000
3,3hJcpXnkhUx7DBjb,-0.446446,-0.947432,-2.301631,-0.630058,-0.540918,-0.490731,-0.530960,-0.478637,-0.470811,...,-0.622986,-0.565070,-0.479899,-0.475912,-0.145865,-0.145865,2.257474,-0.394317,-0.492601,0.000000
4,5KkPKkBVIA4MkSiG,2.869514,-0.993950,0.010859,-0.415917,-0.464674,-0.409800,-0.446892,-0.398660,-0.470811,...,-0.778196,-0.763623,-0.479899,-0.475912,-0.145865,-0.145865,-0.136657,-0.394302,0.039906,0.000000
5,8W3c4P6CZQQieI7A,-0.446446,0.981444,0.609356,-0.485337,-0.451349,-0.412184,-0.449369,-0.408335,-0.426040,...,-0.764086,-0.729793,0.464164,0.275528,-0.145865,-0.145865,-0.550907,-0.394317,-0.492601,0.000000
6,9Te1utCTmj7gTQM9,-0.446446,-0.498911,0.461132,-0.267644,-0.169379,-0.293541,-0.326128,-0.478637,-0.470811,...,-0.270237,-0.267458,1.785852,1.327543,-0.145865,-0.145865,0.290203,0.488118,-0.492601,0.142857
7,A1g7xnOGcmYwkZbc,-0.446446,1.088274,0.801320,0.181289,0.203090,0.319369,0.447612,0.827742,1.106176,...,0.788012,0.837912,-0.479899,-0.475912,-0.145865,-0.145865,-0.550907,-0.394317,-0.264383,0.000000
8,BG4qfLXo8chh5tIY,2.869514,0.911745,0.553144,-0.608940,-0.408487,-0.360469,-0.398227,-0.239479,-0.346057,...,-0.622986,-0.567139,-0.479899,-0.475912,-0.145865,-0.145865,-0.439302,-0.277229,0.106470,0.000000
9,BZvkXIEsrZBFI7E6,-0.446446,-1.375831,-1.058158,0.651648,-0.500608,-0.490731,-0.530960,-0.478637,-0.470811,...,-0.622986,-0.698844,2.352290,1.778407,-0.145865,-0.145865,1.828817,2.102325,-0.492601,0.200000


In [230]:
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(master_df[standardized_columns])

KMeans(n_clusters=4, random_state=42)

In [231]:
score = silhouette_score(master_df[standardized_columns], kmeans.labels_, metric='euclidean')
score

np.float64(0.20363424812030795)

In [232]:
predictions = kmeans.predict(master_df[standardized_columns])

In [233]:
master_df['cluster'] = predictions

In [234]:
def cluster_summary(row):
    if (row['cluster'] == 0):
        return 'Accounts eligible for Credit Line Increase without Risk'
    elif (row['cluster'] == 1):
        return 'Non-performing accounts that pose a high risk'
    elif (row['cluster'] == 2):
        return 'Accounts eligible for Credit Line Increase but has risk of potential defaults or fraud'
    else:
        return 'No Credit Line Increase required'

In [235]:
master_df['customer_segmentation'] = master_df.apply(cluster_summary, axis=1)

In [236]:
master_df

,current_account_nbr,external_status_reason_code,cu_bhv_scr,cu_crd_bureau_scr,mo_tot_sales_array_1,mo_tot_sales_array_2,mo_tot_sales_array_3,mo_tot_sales_array_4,mo_tot_sales_array_5,mo_tot_sales_array_6,...,cu_nbr_days_dlq,ca_max_dlq_lst_6_mnths,ca_nsf_count_lst_12_months,return_check_cnt_total,gross_fraud_amt,net_fraud_amt,transaction_return_cnt,first_purchase_ind,cluster,customer_segmentation
0,0ctPdTAguBbpQDMF,-0.446446,-1.375831,0.645648,-0.621081,-0.531715,-0.490731,-0.530960,-0.478637,-0.470811,...,2.352290,1.778407,-0.145865,-0.145865,-0.534239,-0.394317,-0.492601,1.000000,0,Accounts eligible for Credit Line Increase wit...
1,2myqqt3xp0AAjR3b,-0.446446,-0.903619,0.918791,-0.010173,-0.204622,-0.173141,-0.530960,-0.478637,-0.470811,...,3.768384,2.905567,-0.145865,-0.145865,1.164958,0.416182,0.465912,0.100000,2,Accounts eligible for Credit Line Increase but...
2,3Hj6EuxP2uay1tZR,-0.446446,1.055008,-2.301631,-0.630058,-0.540918,-0.490731,-0.530960,-0.478637,-0.470811,...,-0.479899,-0.475912,-0.145865,-0.145865,-0.214070,-0.394317,-0.492601,0.000000,0,Accounts eligible for Credit Line Increase wit...
3,3hJcpXnkhUx7DBjb,-0.446446,-0.947432,-2.301631,-0.630058,-0.540918,-0.490731,-0.530960,-0.478637,-0.470811,...,-0.479899,-0.475912,-0.145865,-0.145865,2.257474,-0.394317,-0.492601,0.000000,2,Accounts eligible for Credit Line Increase but...
4,5KkPKkBVIA4MkSiG,2.869514,-0.993950,0.010859,-0.415917,-0.464674,-0.409800,-0.446892,-0.398660,-0.470811,...,-0.479899,-0.475912,-0.145865,-0.145865,-0.136657,-0.394302,0.039906,0.000000,0,Accounts eligible for Credit Line Increase wit...
5,8W3c4P6CZQQieI7A,-0.446446,0.981444,0.609356,-0.485337,-0.451349,-0.412184,-0.449369,-0.408335,-0.426040,...,0.464164,0.275528,-0.145865,-0.145865,-0.550907,-0.394317,-0.492601,0.000000,0,Accounts eligible for Credit Line Increase wit...
6,9Te1utCTmj7gTQM9,-0.446446,-0.498911,0.461132,-0.267644,-0.169379,-0.293541,-0.326128,-0.478637,-0.470811,...,1.785852,1.327543,-0.145865,-0.145865,0.290203,0.488118,-0.492601,0.142857,2,Accounts eligible for Credit Line Increase but...
7,A1g7xnOGcmYwkZbc,-0.446446,1.088274,0.801320,0.181289,0.203090,0.319369,0.447612,0.827742,1.106176,...,-0.479899,-0.475912,-0.145865,-0.145865,-0.550907,-0.394317,-0.264383,0.000000,3,No Credit Line Increase required
8,BG4qfLXo8chh5tIY,2.869514,0.911745,0.553144,-0.608940,-0.408487,-0.360469,-0.398227,-0.239479,-0.346057,...,-0.479899,-0.475912,-0.145865,-0.145865,-0.439302,-0.277229,0.106470,0.000000,0,Accounts eligible for Credit Line Increase wit...
9,BZvkXIEsrZBFI7E6,-0.446446,-1.375831,-1.058158,0.651648,-0.500608,-0.490731,-0.530960,-0.478637,-0.470811,...,2.352290,1.778407,-0.145865,-0.145865,1.828817,2.102325,-0.492601,0.200000,2,Accounts eligible for Credit Line Increase but...


# Classification with K-Means Clustering
Cluster 0: Accounts eligible for Credit Line Increase without Risk

Cluster 1: Non-performing accounts that pose a high risk

Cluster 2: Accounts eligible for Credit Line Increase but has risk of potential defaults or fraud 

Cluster 3: No Credit Line Increase required

Current silhouette score: 0.2

# Next Steps

Check with PCA and apiori algorithm

-- try to incorporate above to increase the silhouette score and make more accurate clusters/higher score